<CENTER>
</br>
<p><font size="5"> Nathan Sanglier </span></p>
<p><font size="4">  Global Optimization </font></p>
<p></p>
<p><font size="5"> Metaheuristics for Constrained Optimization </font></p>
</p></br>
</p>
</CENTER>

----------------------------

## <center>  <span style="color:#FF0000"> NOT FINISHED YET </span> </center>

##  <span style="color:#00B8DE"> 0 - Introduction </span>

> The goal of this notebook is to test 3 very popular techniques in global optimization on a non-convex function with variables' domains constraints: Particle Swarm Optimization, Differential Evolution, and Simulated Annealing. If you want to see the plots and LaTeX equations appear well, please download and run the notebook.

In [ ]:
using Plots
using LinearAlgebra: norm
using Statistics
using Random

# <span style="color:#00B8DE"> I - Particle Swarm Optimization (PSO)</span><a name="PSO"></a>
    
[Table of contents](#table-of-contents)

> We consider the function $f(x,y)=x^2+y^2+15(\cos(x-1)+\sin(y+1))$, with $(x,y)\in\mathcal{E}=[-10,10]\times [-10,10]$.
>- We will implement particle swarm optimization with $n=10$ particles. Initial positions are randomly selected in $\mathcal{E}$. Speed equation and speed updates are given by
\begin{equation}
\begin{array}{ll}
v_{i,t+1} &= \alpha v_{i,t}+\beta y_{i,t}(x^{opt}_{i,t}-x_{i,t})+\gamma z_{i,t}(x^{opt}_{t}-x_{i,t})\\
x_{i,t+1} &= \Pi_{\mathcal{E}}(x_{i,t}+v_{i,t+1})
\end{array}
\end{equation}
where $\alpha,\beta,\gamma$ are constants, $x^{opt}_{i,t}$ and $x^{opt}_{t}$ are the best solutions obtained so far for particle $i$ and the whole set of particles and   $y_{i,t},z_{i,t}\sim\mathcal{U}_{[0,1]}$ introduce randomization of move lengths.
$\Pi_{\mathcal{E}}$ is the projection on the domain of constraints.

> Let us plot the function

In [ ]:
f(x,y) = x^2 + y^2 + 15 * (cos(x-1) + sin(y+1))

# u = [x, y]
f(u) = u[1]^2 + u[2]^2 + 15 * (cos(u[1]-1) + sin(u[2]+1));

In [ ]:
opt_f = [-1.8872281420673982, -2.2641283236123844];

In [ ]:
println("Min f(x, y) = $(f(opt_f))")
println("Optimum at (x, y)* = ($(opt_f[1]), $(opt_f[2]))")

In [ ]:
L = 10
dx_f, dy_f = range(-L, L, 201), range(-L, L, 201)
dz_f = f.(dx_f', dy_f)

contourf(dx_f, dy_f, dz_f, nlevels = 40, c=:dense, size=(500, 350))
scatter!([opt_f[1]], [opt_f[2]], label="opt")

> Let us implement particle swarm optimization with $n=10$ particles. Initial positions are randomly selected in $\mathcal{E}$. Speed equation and speed updates are given by
\begin{equation}
\begin{array}{ll}
v_{i,t+1} &= \alpha v_{i,t}+\beta y_{i,t}(x^{opt}_{i,t}-x_{i,t})+\gamma z_{i,t}(x^{opt}_{t}-x_{i,t})\\
x_{i,t+1} &= \Pi_{\mathcal{E}}(x_{i,t}+v_{i,t+1})
\end{array}
\end{equation}
where $\alpha,\beta,\gamma$ are constants, $x^{opt}_{i,t}$ and $x^{opt}_{t}$ are the best solutions obtained so far for particle $i$ and the whole set of particles and   $y_{i,t},z_{i,t}\sim\mathcal{U}_{[0,1]}$ introduce randomization of move lengths.
$\Pi_{\mathcal{E}}$ is the projection on the domain of constraints.
Let us select for instance $\alpha,\beta,\gamma=.9,.5,.5$ and possibly test other choices.

In [ ]:
function find_best_particle_init(p_min)     # Utility function to find the best particle (i.e. the one for which the objective value is the lowest)
    
    f_values = zeros(n_part)
    for i in 1:n_part
        f_values[i] = f(p_min[:, i])
    end

    ind_best = argmin(f_values)
    best_particle = p_min[:, ind_best]
    return best_particle
end;

In [ ]:
function project_to_domain(particle, L)     # Utility function to project a particle position on the domain
    
    return max.(min.(particle, L), -L)      # if one of its corrdinates is beyond L, it is set to L and if it is beyond -L, it is set to -L
end;

In [ ]:
# Particle Swarm Optimization Algorithm
function particle_swarm_optim(f, init_pos, init_v, n_iter, L, α, β, γ)

    n_dims = size(init_pos)[1]
    n_part = size(init_pos)[2]

    pos = zeros(n_dims, n_part, n_iter)                 # Note we will store all the positions for each iteration
    pos[:, :, 1] = init_pos
    
    v = init_v                                          # Speed of particles

    p_min = pos[:, :, 1]                                # Obviously, at initialization, the best position obtained so far for each particle is the one at initialization
    path_p_min = zeros(n_dims, n_part, n_iter)          # We store the path of best position for each particle for each iteration
    path_p_min[:, :, 1] = p_min
    
    p_min_glob = find_best_particle_init(p_min)         # The best position obtained at initialization accross all particles
    path_p_min_glob = zeros(n_dims, n_iter)             # We store the path of best position accross all particles for each iteration
    path_p_min_glob[:, 1] = p_min_glob

    t = 1

    while t < n_iter
        
        r_p = repeat(rand(n_part), 1, 2)'   # Introducing Randomization for the β part
        r_g = repeat(rand(n_part), 1, 2)'   # Introducing Randomization for the γ part

        v = α * v + β *  r_p .* (p_min - pos[:, :, t]) + γ * r_g .* (repeat(p_min_glob, 1, n_part) - pos[:, :, t])     # Update of speed coordinate for each particle

        for i in 1:n_part
            pos[:, i, t+1] = project_to_domain(pos[:, i, t] + v[:, i], L)   # Update of position coordinate for each particle
            
            if f(pos[:, i, t+1]) < f(p_min[:, i])       # If the new position calculated yields to a lower value of f than the best one for this particle so far
                
                p_min[:, i] = pos[:, i, t+1]            # Then the new position is now the best one for this particle

                if f(p_min[:, i]) < f(p_min_glob)       # If this new position is the best one for this particle and is better than the best position accross all particules
                    
                    p_min_glob = p_min[:, i]            # Then this new position is now the best one accross all particles
                end
            end

            path_p_min[:, i, t+1] = p_min[:, i]
        end
        
        path_p_min_glob[:, t+1] = p_min_glob

        t = t + 1
    end

    return pos, path_p_min, path_p_min_glob
end;

In [ ]:
α, β, γ = 0.9, 0.5, 0.5     # Paramaeters of the algorithm

n_dims = 2      # number of dimensions of the input vector of our function to minimize
n_part = 10     # number of particles
n_iter = 200    # number of iterations

init_pos = L * (2 * rand(n_dims, n_part) .- 1)      # at initialization, each position coordinate of each particle is a realization of a uniform law between -10 and 10
init_v = L * (2 * rand(n_dims, n_part) .- 1);       # at initialization, each speed coordinate of each particle is a realization of a uniform law between -10 and 10

In [ ]:
pos, path_p_min, path_p_min_glob = particle_swarm_optim(f, init_pos, init_v, n_iter, L, α, β, γ);

> Let us plot the trajectory of a few particles, plot the evolution of the coordinates of $x^{opt}_{t}$, plot the particles cloud at some instants, and plot the evolution of the standard deviation and RMSE of the particles cloud.

In [ ]:
graph = contourf(dx_f, dy_f, dz_f, nlevels = 40, c=:dense, size=(800, 600))
scatter!([opt_f[1]], [opt_f[2]], label="opt", title="Paths of Particles")
for i in 2:4
    plot!(pos[1, i, :], pos[2, i, :], marker=:circle, label="Path Position Particle $i")
end
plot!(path_p_min_glob[1, :], path_p_min_glob[2, :], marker=:square, markersize=4, c=:"black", label="Path Best Position Overall")
graph

<span style="color:#DAF7A6">
We can see that the few particles plotted converge towards the optimum. At the first iterations, the particles make big movements on the graph that enable diversification, i.e. we tend to converge towards the global minima and not a local minima. This diversification is due to the fact that for each particle, we do not force the particle to keep its best position so far, but we just store the best position accross all particules at each iteration.</br>
If we plot the path of the best position accross all particles at each iteration, we can see it converges very fast to the optimum.

In [ ]:
L_graphs = []
obs_moments = round.(Int, range(1, n_iter, length=4))

for t in obs_moments
    graph = contourf(dx_f, dy_f, dz_f, nlevels = 40, c=:dense, size=(200, 150))
    scatter!([opt_f[1]], [opt_f[2]], label="opt")
    for i in 1:n_part
        scatter!([pos[1, i, t]], [pos[2, i, t]], c=:"green", label="")
    end
    scatter!([path_p_min_glob[1, t]], [path_p_min_glob[2, t]], marker=:square, c=:"black", label="Current Best Position at iter $t")
    push!(L_graphs, graph)
end
plot(L_graphs..., size=(1000, 750), title="Particles Clouds")

<span style="color:#DAF7A6">
We can see that as the number of iterations increase the dispersion of particles is more reduced and centered around the optimum. Moreover, the best position accross all particles converges rapidly to the optimum.

In [ ]:
dist_matrix = zeros(n_part, n_iter)
array_std = zeros(n_iter)

for t in 1:n_iter
    centroid = mean(pos[:, :, t], dims=n_dims)
    for i in 1:n_part
        dist_matrix[i, t] = norm(pos[:, i, t] - centroid)
    end
    array_std[t] = std(dist_matrix[:, t])
end

plot(array_std, label="", title="Stdev of particles distances w.r.t. centroid")
xlabel!("Nb of iterations")
ylabel!("Stdev")

<span style="color:#DAF7A6">
As expected, we can see that the cloud of particles is more and more close to its centroïd as the number of iterations increase (i.e. the standard deviation of distance of the particules to the cloud centroid (average position) decreases to 0). The fluctuations at the first iterations are due to the diversification mechanism explained precedently.

In [ ]:
dist_matrix = zeros(n_part, n_iter)
array_rmse = zeros(n_iter)

for t in 1:n_iter
    for i in 1:n_part
        dist_matrix[i, t] = norm(pos[:, i, t] - opt_f)
    end
    array_rmse[t] = mean(dist_matrix[:, t])
end

plot(array_rmse, label="", title="RMSE of Particles Distance w.r.t. Optimum")
xlabel!("Nb of iterations")
ylabel!("RMSE")

<span style="color:#DAF7A6">
As expected, we can see that the cloud of particles converges to the optimum, since the RMSE of particles distance to optimum tends to 0. The fluctuations at the first iterations are due to the diversification mechanism explained precedently.

# <span style="color:#00B8DE"> II - Differential Evolution (DE)</span><a name="DE"></a>

Let us implement differential evolution algorithm.

In [ ]:
function differential_evol_optim(f, init_pos, n_iter, L, CR, F)

    n_dims = size(init_pos)[1]
    n_part = size(init_pos)[2]

    pos = zeros(n_dims, n_part, n_iter)         # Note we will store all the positions for each iteration
    pos[:, :, 1] = init_pos
    current_pos = init_pos                      # We store the current position for each particle
    
    p_min_glob = find_best_particle_init(init_pos)          # The best position obtained at initialization accross all particles
    path_p_min_glob = zeros(n_dims, n_iter)                 # We store the path of best position accross all particles for each iteration
    path_p_min_glob[:, 1] = p_min_glob

    t = 1

    while t < n_iter

        for i in 1:n_part
            
            neighbors = [k for k in 1:n_part if k != i]         # Particles that can be selected to update the ith particle (i.e. all excluded the ith)
            shuffle_inds = randperm(length(neighbors))          
            neighbors_selec = neighbors[shuffle_inds[1:3]]      # We pick 3 particles among the possible ones randomly

            dim_selec = rand(1:n_dims)      # dimension we select randomly for the crossover

            new_pos = zeros(n_dims)         # Potential new position of the particle

            for j in 1:n_dims

                r = rand()
                if r < CR || j == dim_selec     # If crossover rate is above r and the dimension is the one selected above, we do crossover
                    
                    new_pos[j] = current_pos[j, neighbors_selec[1]] + F * (current_pos[j, neighbors_selec[2]] - current_pos[j, neighbors_selec[3]])
                    
                else
                    new_pos[j] = current_pos[j, i]
                end
            end

            if f(new_pos) < f(current_pos[:, i])            # If the potential new position is better than the current one, we update its current position
                current_pos[:, i] = new_pos

                if f(current_pos[:, i]) < f(p_min_glob)     # If this new position is better than the best one found so far among all particles
                    
                    p_min_glob = current_pos[:, i]          # Then we update this best position found so far among all particles
                end
            end

            pos[:, i, t+1] = current_pos[:, i]
        end

        path_p_min_glob[:, t+1] = p_min_glob

        t += + 1
    end

    return pos, path_p_min_glob
end;

In [ ]:
CR, F = 0.9, 0.8 # crossover probability and differential weight

n_dims = 2      # number of dimensions of the input vector of our function to minimize
n_part = 10     # number of particles
n_iter = 50     # number of iterations

init_pos = L * (2 * rand(n_dims, n_part) .- 1);      # at initialization, each position coordinate of each particle is a realization of a uniform law between -10 and 10

In [ ]:
pos, path_p_min_glob = differential_evol_optim(f, init_pos, n_iter, L, CR, F);

In [ ]:
graph = contourf(dx_f, dy_f, dz_f, nlevels = 40, c=:dense, size=(800, 600))
scatter!([opt_f[1]], [opt_f[2]], label="opt")
for i in 3:6
    plot!(pos[1, i, :], pos[2, i, :], marker=:circle, label="Path Position Particle $i")
end
plot!(path_p_min_glob[1, :], path_p_min_glob[2, :], marker=:square, markersize=4, c=:"black", label="Path Best Position Overall")
graph

<span style="color:#DAF7A6">
We can see here with the differential evolution algorithm, that the few particles plotted converge towards the optimum. At the first iterations, the particles make big movements on the graph that enable diversification with the mechanism of a probability of crossover with an other particle.</br>
If we plot the path of the best position accross all particles at each iteration, we can see it converges very fast to the optimum.

In [ ]:
L_graphs = []
obs_moments = round.(Int, range(1, n_iter, length=4))

for t in obs_moments
    graph = contourf(dx_f, dy_f, dz_f, nlevels = 40, c=:dense, size=(200, 150))
    scatter!([opt_f[1]], [opt_f[2]], label="opt")
    for i in 1:n_part
        scatter!([pos[1, i, t]], [pos[2, i, t]], c=:"green", label="")
    end
    scatter!([path_p_min_glob[1, t]], [path_p_min_glob[2, t]], marker=:square, c=:"black", label="Current Best Position at iter $t")
    push!(L_graphs, graph)
end
plot(L_graphs..., size=(1000, 750))

<span style="color:#DAF7A6">
We can see that as the number of iterations increase the dispersion of particles is more reduced and centered around the optimum. Moreover, the best position accross all particles converges rapidly to the optimum.

In [ ]:
dist_matrix = zeros(n_part, n_iter)
array_std = zeros(n_iter)

for t in 1:n_iter
    centroid = mean(pos[:, :, t], dims=n_dims)
    for i in 1:n_part
        dist_matrix[i, t] = norm(pos[:, i, t] - centroid)
    end
    array_std[t] = std(dist_matrix[:, t])
end

plot(array_std, label="", title="Stdev of Particles Positions")
xlabel!("Nb of iterations")
ylabel!("Stdev")

<span style="color:#DAF7A6">
As expected, we can see that the cloud of particles is more and more close to its centroïd as the number of iterations increase (i.e. the standard deviation of distance of the particules to the cloud centroid (average position) decreases to 0). The fluctuations at the first iterations are due to the diversification mechanism explained precedently.

In [ ]:
dist_matrix = zeros(n_part, n_iter)
array_rmse = zeros(n_iter)

for t in 1:n_iter
    for i in 1:n_part
        dist_matrix[i, t] = norm(pos[:, i, t] - opt_f)
    end
    array_rmse[t] = mean(dist_matrix[:, t])
end

plot(array_rmse, label="", title="RMSE of Particles Distance w.r.t. Optimum")
xlabel!("Nb of iterations")
ylabel!("RMSE")

<span style="color:#DAF7A6">
As expected, we can see that the cloud of particles converges to the optimum, since the RMSE of particles distance to optimum tends to 0.

# <span style="color:#00B8DE"> III - Simulated annealing (SA)</span><a name="DE"></a>

In [ ]:
function SA(f, x0; σ=1, K=10^3, Ti=10, Tf=1e-2)
    #=
    - simulated annealing algorithm
    - inputs
        f: function to minimize
        x0: initial position
        σ: std of moves 
        K: number of iterations
        Ti: initial temperature
        Tf: final temperature
    - outputs
        points: sequence of tested points
    =#
    α      = exp(log(Tf/Ti)/K)
    tt     = zeros(K) # temperatures
    xx     = zeros(K) # positions
    yy     = zeros(K) # values of the objective
    tt[1]  = Ti       # initial temperature
    xx[1]  = x0       # initial position
    yy[1]  = f(x0)    # initial value of the objective

    for k=2:K
        tt[k]  = tt[k-1]*α
        x      = xx[k-1] + σ*randn()
        y      = f(x)
        if (rand() < min(1,exp(-(y-yy[k-1])/tt[k])))
            xx[k] = x 
            yy[k] = y
        else 
            xx[k] = xx[k-1] 
            yy[k] = yy[k-1]           
        end
    end

    xx,yy
end;

In [ ]:
# function to optimize
f(x)= x^2 - 5cos(2π*x)  #function to be minimized

# SA algorithm
xx,yy = SA(f, randn()); # points visited by the algorithm

In [ ]:
using Plots
pos = -5:.01:5
p1 = plot(pos,f.(pos),label="f(x)")
p1 = scatter!(xx,yy,ms=2, label="successive positions",legend=:top) 
p2 = plot(xx,label="successive x values")
plot(p1,p2,layout=(1,2),size=(1000,400))

<span style="color:#DAF7A6">

Simulated Annealing is a metaheuristic used to solve optimization problems. Its name comes from annealing in metallurgy, a technique involving heating and controlled cooling of a material to alter its physical properties

This notion of slow cooling implemented in the simulated annealing algorithm is interpreted as a slow decrease in the temperature-dependent probability of accepting worse solutions as the solution space is explored. Accepting worse solutions allows for a more extensive search for the global optimal solution, which is important during the first iterations.

The decrease of the probability of accepting worse solutions is done by the coefficient $\alpha$ : at each iteration, we multiply the current temperature by $\alpha = exp(log(T_f/T_i)/K)$. $T_i$ is the initial temperature and $T_f$ the final temperature. Since K is the number of iterations, we can see that the current temperature at the least iteration is indeed $T_f$.

A potential new position is calculated from the current one thanks with a standard deviation given. Then this position is accepted for sure if it leads to a smaller value of the objective function (exp(-(y-yy[k-1])/tt[k]) > 1). But the position can be also accepted even it is worse, with a probability exp(-(y-yy[k-1])/tt[k]). As mentionned above, this probability decreases through time since it depends on the current temperature (which is decreasing at each iteration) at the denominator of the exponential. It also decreases as the solution is bad compared to the current one.


On the graphs, we can see that after the diversification phase during the first iterations, the algorithm converges towards the optimum in the next iterations.

> Let us test the influence of parameters.

<span style="color:#DAF7A6">
Let's test the influence of σ on the algorithm.

In [ ]:
σ_values = [0.1, 1, 10]
L_graphs = []

for σ_val in σ_values
    xx,yy = SA(f, randn(); σ=σ_val, K=10^3, Ti=10, Tf=1e-2)
    graph = plot(xx, label="σ=$σ_val")
    push!(L_graphs, graph)
end

plot(L_graphs..., layout=(1,length(σ_values)),size=(1000,400), title="Successive x values")

<span style="color:#DAF7A6">

We can see that if $\sigma$ is too small, then we will do too little steps and we will focus only on one region of the graph. This can be dangerous as we could not explore the area where the optimum is. In the opposite, when $\sigma$ is too high, then we do too big steps and there is too much diversification, it can be dangerous as we could not converge to the optimum because we "jump over its position".

<span style="color:#DAF7A6">
We won't study the influence of the number of iterations on the algorithm as there is little interest since it is only related to the time we get to converge the the optimum, with a too little number of iterations we could not have the time to converge, wheereas with a too big number, some iterations would be useless.

<span style="color:#DAF7A6">

Let's test the influence of $T_i$ (initial temperature) on the algorithm.

In [ ]:
Ti_values = [0.1, 10, 100]
L_graphs = []

for ti_val in Ti_values
    xx,yy = SA(f, randn(); σ=1, K=10^3, Ti=ti_val, Tf=1e-2)
    graph = plot(xx, label="Ti=$ti_val")
    push!(L_graphs, graph)
end

plot(L_graphs..., layout=(1,length(σ_values)),size=(1000,400), title="Successive x values")

<span style="color:#DAF7A6">

Here, it is difficult to see well the impact of $T_i$ on the convergence of the algorithm. But we can say that the higher $T_i$ is (compared to a fixed $T_f$), the more brutal will be the changes in the temperature and so the changes in the probability of accepting worse solutions. If $T_i$ is too close to $T_f$, then the convergence can be faster but it could prevent from diversification and we could be stuck in a local optima. If $T_i$ is too far from $T_f$, then the convergence can be slower because of too much diversification in the first iterations.

<span style="color:#DAF7A6">

We don't need to study the impact of $T_f$ (final temperature), as the conclusions will be similar to $T_i$ but in the other way (the closer $T_f$ is from $T_i$, the less brutal the changes in temperature and thus in probability of accpeting worse solution).